#### Vacuum Agent Path Planning

Help your vacuum agent go home as efficiently as possible!
While the agent is in explore mode, it records every position it visits, then when it needs to go home, it computes the shortest path from its current position to [1,1] that goes only through positions it visited.  You are going to write a function
<pre>
agent_path_home(list_of_positions)  => path
</pre>
where the return path is also a list of positions, each of which appears in the input list.   The first element of the path must be the last element in the position list (i.e. where the agent is starting from) and the last element of the path must be [1,1] and any two elements of the path next to each other must be adjacent coordinates.  The path returned must be the shortest possible path.

The next cell contains several examples.  

In [ ]:
input1 = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 17), (2, 17), (3, 17), (4, 17), (5, 17), (6, 17), (6, 18), (5, 18), (4, 18), (4, 17), (4, 16), (4, 15), (4, 14), (4, 13), (4, 12), (4, 11), (4, 10), (4, 9), (4, 8), (4, 7), (5, 7), (5, 8), (5, 9), (6, 9), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (6, 13), (7, 13), (8, 13), (9, 13), (10, 13), (11, 13), (12, 13), (13, 13), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (12, 18)]
shortestpath1 = [(13, 18), (13, 17), (13, 16), (13, 15), (13, 14), (13, 13), (12, 13), (11, 13), (10, 13), (9, 13), (8, 13), (7, 13), (7, 12), (7, 11), (7, 10), (7, 9), (6, 9), (5, 9), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (3, 17), (2, 17), (1, 17), (1, 16), (1, 15), (1, 14), (1, 13), (1, 12), (1, 11), (1, 10), (1, 9), (1, 8), (1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1)]
 

In [ ]:
input2 = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 4), (2, 4), (2, 3), (2, 2), (2, 1), (3, 1), (3, 2), (3, 3), (3, 2), (4, 2), (5, 2), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (5, 6), (4, 6), (3, 6), (3, 5), (4, 5), (4, 4), (5, 4), (5, 3), (5, 2), (5, 1), (5, 2), (5, 3), (5, 4), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (3, 9), (2, 9), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (2, 18), (1, 18), (1, 17), (1, 16), (1, 17), (2, 17), (3, 17), (3, 18), (3, 17), (3, 16), (4, 16)]
shortestpath2 = [(3, 16), (3, 15), (3, 14), (3, 13), (3, 12), (3, 11), (3, 10), (3, 9), (4, 9), (4, 8), (4, 7), (4, 6), (4, 5), (4, 4), (5, 4), (5, 3), (5, 2), (4, 2), (3, 2), (2, 2), (1, 2), (1, 1)]


In [ ]:
input3 = [(1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (2, 4), (1, 4), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (4, 8), (4, 7), (4, 6), (4, 5), (4, 4), (4, 3), (4, 2), (5, 2), (5, 1), (6, 1), (7, 1), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (7, 5), (6, 5), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 4), (12, 4), (13, 4), (13, 5), (13, 4), (13, 3), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2), (17, 2), (17, 1), (18, 1), (18, 2), (17, 2), (17, 3), (17, 4), (17, 5), (17, 6), (17, 7), (17, 8)]
shortestpath3 = [(17, 7), (17, 6), (17, 5), (17, 4), (17, 3), (17, 2), (16, 2), (15, 2), (14, 2), (13, 2), (13, 3), (13, 4), (12, 4), (11, 4), (10, 4), (9, 4), (8, 4), (7, 4), (6, 4), (5, 4), (4, 4), (3, 4), (3, 3), (3, 2), (3, 1), (2, 1), (1, 1)]


In [ ]:
input4 = [(1, 1), (1, 2), (1, 3), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (6, 3), (6, 4), (6, 5), (5, 5), (5, 4), (5, 5), (6, 5), (7, 5), (7, 6), (7, 5), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5), (5, 5), (5, 4), (6, 4), (6, 5)]
shortestpath4 = [(6, 4), (6, 3), (6, 2), (5, 2), (4, 2), (3, 2), (2, 2), (1, 2), (1, 1)]